In [ ]:
import xml.etree.ElementTree as ET
import json
import re

def safe_float(text):
    """Convert a string to float after removing commas."""
    if text is None:
        return 0.0
    return float(re.sub(r",", "", text.strip()))

def parse_decision_logs(root, hero):
    """
    Parse the XML tree and, for every opponent decision (action) in rounds 1 and later
    (preflop and beyond), produce a snapshot log capturing the game state at the moment
    BEFORE the action is processed.
    
    In this snapshot:
      - The snapshot does NOT include any actions from round 0 (blinds/antes).
      - The re-indexed action counter is incremented for every action in rounds ≥ 1,
        so that no action gets a number of 0.
      - Only opponent actions are logged (the hero's actions are skipped).
      - The snapshot shows the acting opponent's hole cards (if available); the hero's
        or other players' hole cards are not revealed in the snapshot.
    """
    logs = []

    for game in root.findall('game'):
        general = game.find('general')
        if general is None:
            continue

        # --- Extract blinds and ante ---
        small_blind_elem = general.find('smallblind')
        big_blind_elem = general.find('bigblind')
        ante_elem = general.find('ante')
        small_blind = safe_float(small_blind_elem.text) if small_blind_elem is not None else 0.0
        big_blind = safe_float(big_blind_elem.text) if big_blind_elem is not None else 0.0
        ante = safe_float(ante_elem.text) if ante_elem is not None else 0.0

        # --- Process players ---
        players = {}        # key: player name, value: player info
        active_players = {} # tracks whether a player is still in the hand
        players_elem = general.find('players')
        if players_elem is None:
            continue

        for player_elem in players_elem.findall('player'):
            name = player_elem.attrib.get('name')
            seat = int(player_elem.attrib.get('seat'))
            is_dealer = (player_elem.attrib.get('dealer') == "1")
            chips = safe_float(player_elem.attrib.get('chips'))
            bet = safe_float(player_elem.attrib.get('bet'))
            win = safe_float(player_elem.attrib.get('win'))
            players[name] = {
                'seat': seat,
                'is_dealer': is_dealer,
                'chips': chips,
                'bet': bet,
                'win': win
            }
            active_players[name] = True

        # --- Determine relative positions ---
        # Sort players by seat number.
        sorted_players = sorted(players.items(), key=lambda item: item[1]['seat'])
        # Find the dealer (button). If none is marked, assume the first player is the dealer.
        dealer_index = None
        for i, (name, info) in enumerate(sorted_players):
            if info['is_dealer']:
                dealer_index = i
                break
        if dealer_index is None:
            dealer_index = 0

        button_player = sorted_players[dealer_index][0]
        small_blind_player = sorted_players[(dealer_index + 1) % len(sorted_players)][0]
        big_blind_player = sorted_players[(dealer_index + 2) % len(sorted_players)][0]

        # Assign relative position codes: small blind = 0, big blind = 1, button = 2, others = 3.
        player_positions = {}
        for name in players:
            if name == small_blind_player:
                players[name]['position_relative'] = 0
            elif name == big_blind_player:
                players[name]['position_relative'] = 1
            elif name == button_player:
                players[name]['position_relative'] = 2
            else:
                players[name]['position_relative'] = 3
            player_positions[name] = players[name]['position_relative']

        # Skip this game if the hero is not present.
        if hero not in players:
            continue

        # --- Initialize state variables ---
        pot_size = 0.0
        cumulative_actions = []  # will contain actions from rounds >= 1 only
        board_cards = []         # community cards seen so far
        pocket_cards = {}        # updated when encountering <cards type="Pocket">
        snapshot_action_counter = 0  # counter for actions in rounds >= 1

        # --- Process rounds in order (sorted by round number) ---
        rounds = game.findall('round')
        rounds = sorted(rounds, key=lambda r: int(r.attrib.get('no')))
        for r in rounds:
            round_no = int(r.attrib.get('no'))
            for elem in r:
                if elem.tag == 'cards':
                    card_type = elem.attrib.get('type')
                    if card_type == 'Pocket':
                        # Record pocket cards for the given player.
                        player = elem.attrib.get('player')
                        text = elem.text.strip() if elem.text else ""
                        cards = text.split()
                        # If cards are hidden (e.g., "X X"), record as unknown.
                        if any(card.upper().startswith("X") for card in cards):
                            pocket_cards[player] = ["unknown", "unknown"]
                        else:
                            pocket_cards[player] = cards
                    else:
                        # Community cards (flop, turn, river)
                        cards = elem.text.split() if elem.text else []
                        board_cards.extend(cards)
                elif elem.tag == 'action':
                    # Build an action dictionary.
                    action_details = {
                        'round': round_no,
                        'player': elem.attrib.get('player'),
                        'action_type': int(elem.attrib.get('type')),
                        'action_sum': safe_float(elem.attrib.get('sum'))
                    }

                    # For actions in round 0 (blinds/antes), update state only (do not record them).
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        # (If desired, you could update pocket_cards from round 0 as well.)
                        continue

                    # For actions in rounds >= 1, first assign a new re-indexed action number.
                    snapshot_action_counter += 1
                    # Copy the action_details and add the new action number.
                    action_details['action_no'] = snapshot_action_counter

                    # If the acting player is not the hero, produce a snapshot BEFORE processing the action.
                    # (The snapshot’s "previous_actions" will be the cumulative_actions so far.)
                    if action_details['player'] != hero:
                        is_button = (action_details['player'] == button_player)
                        actor_cards = pocket_cards.get(action_details['player'], ["unknown", "unknown"])
                        # Build the snapshot.
                        snapshot = {
                            "gamecode": game.attrib.get("gamecode"),
                            "round_no": round_no,
                            "current_street": (
                                "preflop" if round_no == 1 else
                                "flop" if round_no == 2 else
                                "turn" if round_no == 3 else
                                "river" if round_no == 4 else "unknown"
                            ),
                            "blinds": {
                                "small_blind": small_blind,
                                "big_blind": big_blind,
                                "ante": ante
                            },
                            "player_positions": player_positions,
                            "player_stacks": {name: players[name]['chips'] for name in players},
                            "pot_size": pot_size,  # state BEFORE processing this action
                            "board_cards": board_cards.copy(),
                            "previous_actions": cumulative_actions.copy(),  # only round>=1 actions so far
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for active in active_players.values() if active),
                            "is_button": is_button,
                            "actor_hole_cards": pocket_cards.get(action_details['player'], ["unknown", "unknown"])
                        }
                        logs.append(snapshot)

                    # Now update the state: add this action to the cumulative history.
                    cumulative_actions.append(action_details)
                    pot_size += action_details['action_sum']
                    # Mark the actor as inactive if the action is a fold (we assume type==0 means fold).
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False

    return logs

if __name__ == '__main__':
    try:
        # Parse the XML file (adjust 'test.xml' to your filename)
        tree = ET.parse('test.xml')
        root = tree.getroot()

        # Determine the hero's nickname from the session-level <nickname> element.
        session_general = root.find('general')
        if session_general is not None and session_general.find('nickname') is not None:
            hero = session_general.find('nickname').text.strip()
        else:
            hero = "mmpq8j6x8"  # fallback if not found

        # Parse opponent decision snapshots (skipping hero actions and round 0 actions)
        logs = parse_decision_logs(root, hero)
        print(json.dumps(logs, indent=4))
    except ET.ParseError as e:
        print("Error parsing XML file:", e)


## Improve version of extract data from single iPoker HH xml file

In [2]:
#!/usr/bin/env python3
import xml.etree.ElementTree as ET
import json, re

def safe_float(text):
    try:
        # remove any commas or other non-digit/dot characters
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    # Search all rounds for a <cards type="Pocket" player="..."> element.
    # If the text contains "X" (hidden cards) we return ["unknown","unknown"].
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def parse_decision_logs(root, hero):
    logs = []
    # Process every game
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        # extract blinds and ante (if any)
        blinds = {}
        blinds['small_blind'] = safe_float(general.findtext('smallblind', default="0"))
        blinds['big_blind'] = safe_float(general.findtext('bigblind', default="0"))
        blinds['ante'] = safe_float(general.findtext('ante', default="0"))
        
        # Process players. We assume the players are listed in a <players> element.
        players_elem = general.find('players')
        player_positions = {}   # will hold each player’s relative position code
        player_stacks = {}      # the chip stacks at the beginning of the hand
        # sort players by their seat number
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        # determine the dealer: use the one whose dealer attribute equals "1" (or if none, the first one)
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer','0')=='1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        # Create an “order” starting with the player immediately after the dealer (wrapping around).
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        # For instance, assign positions 0,1,...,n-1 in the order determined.
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        # All players start active.
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # history of actions (only rounds >= 1)
        snapshot_action_counter = 0

        # Process each round in the game.
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    # If it is not a Pocket card then we assume community cards.
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'round': round_no,
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum'))
                    }
                    # Skip round 0 actions: update pot only.
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        continue

                    # For rounds >=1: assign a re-indexed action number.
                    snapshot_action_counter += 1
                    action_details['action_no'] = snapshot_action_counter

                    # For opponent actions (non–hero) produce a snapshot BEFORE processing the action.
                    if child.attrib.get('player') != hero:
                        snapshot = {
                            "gamecode": gamecode,
                            "round_no": round_no,
                            "current_street": street_from_round(round_no),
                            "blinds": blinds,
                            "player_positions": player_positions,
                            "player_stacks": player_stacks,
                            "pot_size": pot_size,  # pot _before_ current action
                            "board_cards": board_cards.copy(),
                            "previous_actions": [],
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for v in active_players.values() if v),
                            "is_button": (player_positions.get(child.attrib.get('player')) == len(player_positions)-1),
                            "actor_hole_cards": get_hole_cards(game, child.attrib.get('player')),
                            # NEW FIELDS:
                            "actor_stack_size": player_stacks.get(child.attrib.get('player'), 0.0),
                            "actor_position": player_positions.get(child.attrib.get('player'))
                        }
                        # Add only previous actions that have been recorded (all from rounds>=1).
                        for act in cumulative_actions:
                            act2 = act.copy()
                            act2["player_position"] = player_positions.get(act["player"])
                            snapshot["previous_actions"].append(act2)
                        logs.append(snapshot)
                    
                    # Update running state: add this action to cumulative_actions and update pot.
                    cumulative_actions.append(action_details.copy())
                    pot_size += action_details['action_sum']
                    # If the action is a fold (action_type == 0) then mark the player as inactive.
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False
        # end of game rounds
    return logs

if __name__ == '__main__':
    try:
        tree = ET.parse('test.xml')
        root = tree.getroot()
        # Get the hero name from the session <nickname> element.
        session_general = root.find('general')
        if session_general is not None and session_general.find('nickname') is not None:
            hero = session_general.find('nickname').text.strip()
        else:
            hero = "mmpq8j6x8"  # fallback
        logs = parse_decision_logs(root, hero)
        print(json.dumps(logs, indent=4))
    except ET.ParseError as e:
        print("Error parsing XML file:", e)


[
    {
        "gamecode": "10136070425",
        "round_no": 1,
        "current_street": "preflop",
        "blinds": {
            "small_blind": 10.0,
            "big_blind": 20.0,
            "ante": 0.0
        },
        "player_positions": {
            "GGM3M": 0,
            "Bekocabron": 1,
            "mmpq8j6x8": 2
        },
        "player_stacks": {
            "GGM3M": 500.0,
            "Bekocabron": 500.0,
            "mmpq8j6x8": 500.0
        },
        "pot_size": 30.0,
        "board_cards": [],
        "previous_actions": [
            {
                "round": 1,
                "player": "mmpq8j6x8",
                "action_type": 0,
                "action_sum": 0.0,
                "action_no": 1,
                "player_position": 2
            }
        ],
        "action": {
            "round": 1,
            "player": "GGM3M",
            "action_type": 3,
            "action_sum": 10.0,
            "action_no": 2
        },
        "players_remainin

## Extract iPoker HH from single hand history file(Ready for training model)

In [ ]:
#!/usr/bin/env python3
import xml.etree.ElementTree as ET
import json, re

def safe_float(text):
    try:
        # remove any commas or other non-digit/dot characters
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    # Search all rounds for a <cards type="Pocket" player="..."> element.
    # If the text contains "X" (hidden cards) we return ["unknown","unknown"].
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def parse_decision_logs(root, hero):
    logs = []
    # Process every game
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        # extract blinds and ante (if any)
        blinds = {}
        blinds['small_blind'] = safe_float(general.findtext('smallblind', default="0"))
        blinds['big_blind'] = safe_float(general.findtext('bigblind', default="0"))
        blinds['ante'] = safe_float(general.findtext('ante', default="0"))
        
        # Process players. We assume the players are listed in a <players> element.
        players_elem = general.find('players')
        player_positions = {}   # will hold each player’s relative position code
        player_stacks = {}      # the chip stacks at the beginning of the hand
        # sort players by their seat number
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        # determine the dealer: use the one whose dealer attribute equals "1" (or if none, the first one)
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer','0')=='1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        # Create an “order” starting with the player immediately after the dealer (wrapping around).
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        # For instance, assign positions 0,1,...,n-1 in the order determined.
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        # All players start active.
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # history of actions (only rounds >= 1)
        snapshot_action_counter = 0

        # Process each round in the game.
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    # If it is not a Pocket card then we assume community cards.
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'round': round_no,
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum'))
                    }
                    # Skip round 0 actions: update pot only.
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        continue

                    # For rounds >=1: assign a re-indexed action number.
                    snapshot_action_counter += 1
                    action_details['action_no'] = snapshot_action_counter

                    # For opponent actions (non–hero) produce a snapshot BEFORE processing the action.
                    if child.attrib.get('player') != hero:
                        snapshot = {
                            "gamecode": gamecode,
                            "round_no": round_no,
                            "current_street": street_from_round(round_no),
                            "blinds": blinds,
                            "player_positions": player_positions,
                            "player_stacks": player_stacks,
                            "pot_size": pot_size,  # pot _before_ current action
                            "board_cards": board_cards.copy(),
                            "previous_actions": [],
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for v in active_players.values() if v),
                            "is_button": (player_positions.get(child.attrib.get('player')) == len(player_positions)-1),
                            "actor_hole_cards": get_hole_cards(game, child.attrib.get('player')),
                            # NEW FIELDS:
                            "actor_stack_size": player_stacks.get(child.attrib.get('player'), 0.0),
                            "actor_position": player_positions.get(child.attrib.get('player'))
                        }
                        # Add only previous actions that have been recorded (all from rounds>=1).
                        for act in cumulative_actions:
                            act2 = act.copy()
                            act2["player_position"] = player_positions.get(act["player"])
                            snapshot["previous_actions"].append(act2)
                        logs.append(snapshot)
                    
                    # Update running state: add this action to cumulative_actions and update pot.
                    cumulative_actions.append(action_details.copy())
                    pot_size += action_details['action_sum']
                    # If the action is a fold (action_type == 0) then mark the player as inactive.
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False
        # end of game rounds
    return logs

if __name__ == '__main__':
    try:
        tree = ET.parse('test.xml')
        root = tree.getroot()
        # Get the hero name from the session <nickname> element.
        session_general = root.find('general')
        if session_general is not None and session_general.find('nickname') is not None:
            hero = session_general.find('nickname').text.strip()
        else:
            hero = "mmpq8j6x8"  # fallback
        logs = parse_decision_logs(root, hero)
        # Instead of printing, write to a JSON file:
        with open('test_hand.json', 'w') as outfile:
            json.dump(logs, outfile, indent=4)
        print("Data extraction complete. Saved to test_hand.json")
    except ET.ParseError as e:
        print("Error parsing XML file:", e)


Data extraction complete. Saved to logs.json


## Extract iPoker HH from folder

In [6]:
#!/usr/bin/env python3
import os
import xml.etree.ElementTree as ET
import json
import re

# ------------------------------
# Helper functions
# ------------------------------

def safe_float(text):
    try:
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def simplify_action(action_details, round_no, blinds, pot_before_action):
    """
    Simplify the action into a single action_type string.
    
    Allowed original types:
      - 0 -> fold
      - 3 -> call
      - 4 -> check
      - 5, 7, 23 -> raise
         * Preflop (round 1): using big blind as reference.
             2.0×BB to 2.3×BB → small raise preflop
             2.3×BB to 3.0×BB → mid raise preflop
             3.0×BB to 4.0×BB → big raise preflop
             ≥ 4.0×BB         → all in preflop
         * Postflop (round > 1): using the pot (before the action) as reference.
             < 35% of pot      → small raise postflop
             35% to 70% of pot  → mid raise postflop
             70% to 110% of pot → big raise postflop
             ≥ 110% of pot     → all in postflop
    """
    allowed_types = {0: "fold", 3: "call", 4: "check", 5: "raise", 7: "raise", 23: "raise"}
    orig_type = action_details['action_type']
    if orig_type not in allowed_types:
        return None
    base_action = allowed_types[orig_type]
    new_action = action_details.copy()
    if base_action != "raise":
        new_action["simple_action_type"] = base_action
    else:
        if round_no == 1:
            bb = blinds.get("big_blind", 1)
            ratio = action_details['action_sum'] / bb if bb != 0 else 0
            if 2.0 <= ratio < 2.3:
                new_action["simple_action_type"] = "small raise preflop"
            elif 2.3 <= ratio < 3.0:
                new_action["simple_action_type"] = "mid raise preflop"
            elif 3.0 <= ratio < 4.0:
                new_action["simple_action_type"] = "big raise preflop"
            elif ratio >= 4.0:
                new_action["simple_action_type"] = "all in preflop"
            else:
                new_action["simple_action_type"] = "raise"
        else:
            ratio = action_details['action_sum'] / pot_before_action if pot_before_action > 0 else 0
            if ratio < 0.35:
                new_action["simple_action_type"] = "small raise postflop"
            elif ratio < 0.70:
                new_action["simple_action_type"] = "mid raise postflop"
            elif ratio < 1.10:
                new_action["simple_action_type"] = "big raise postflop"
            else:
                new_action["simple_action_type"] = "all in postflop"
    return new_action

def parse_decision_logs(root, hero):
    logs = []
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        blinds = {
            'small_blind': safe_float(general.findtext('smallblind', default="0")),
            'big_blind': safe_float(general.findtext('bigblind', default="0")),
            'ante': safe_float(general.findtext('ante', default="0"))
        }
        players_elem = general.find('players')
        player_positions = {}
        # Store initial stacks from the XML.
        player_stacks = {}
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer', '0') == '1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # Simplified actions list (each now includes "action_round")
        snapshot_action_counter = 0
        
        # Track contributions from each player (blinds, bets, etc.)
        player_contributions = {name: 0.0 for name in player_positions}
        
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum')),
                        'action_round': round_no
                    }
                    player = action_details['player']
                    contribution = action_details['action_sum']
                    
                    # For round 0 (blinds/antes) update contributions and pot; no snapshot is created.
                    if round_no < 1:
                        player_contributions[player] += contribution
                        pot_size += contribution
                        continue
                        
                    # For rounds ≥ 1, first capture the current pot (before the current action is added)
                    current_pot = pot_size
                    # Actor's current stack = initial stack minus chips already committed.
                    actor_current_stack = player_stacks.get(player, 0.0) - player_contributions.get(player, 0.0)
                    
                    # Simplify action into a single string category.
                    simple_action = simplify_action(action_details, round_no, blinds, current_pot)
                    
                    if simple_action is not None:
                        snapshot_action_counter += 1
                        simple_action['action_no'] = snapshot_action_counter
                        
                        # Compute current stack for every player.
                        current_player_stacks = {p: player_stacks[p] - player_contributions.get(p, 0.0)
                                                 for p in player_stacks}
                        
                        # Create snapshot only for non-hero actions.
                        if player != hero:
                            snapshot = {
                                "gamecode": gamecode,
                                "round_no": round_no,
                                "current_street": street_from_round(round_no),
                                "blinds": blinds,
                                "player_positions": player_positions,
                                "player_stacks": current_player_stacks,
                                "pot_size": current_pot,
                                "board_cards": board_cards.copy(),
                                "previous_actions": [],
                                "action": simple_action.copy(),
                                "players_remaining": sum(1 for v in active_players.values() if v),
                                "is_button": (player_positions.get(player) == len(player_positions)-1),
                                "actor_hole_cards": get_hole_cards(game, player),
                                "actor_stack_size": actor_current_stack,
                                "actor_position": player_positions.get(player)
                            }
                            for act in cumulative_actions:
                                act2 = act.copy()
                                act2["player_position"] = player_positions.get(act["player"])
                                snapshot["previous_actions"].append(act2)
                            logs.append(snapshot)
                    
                    # Now update contributions and pot AFTER snapshot creation.
                    player_contributions[player] += contribution
                    pot_size += contribution
                    
                    if simple_action is not None:
                        cumulative_actions.append(simple_action)
                    
                    # Mark player as inactive if they folded.
                    if action_details.get('action_type') == 0:
                        active_players[player] = False
    return logs

# ------------------------------
# Process All XML Files in ipoker_hh Folder
# ------------------------------

def process_all_hand_history(root_folder, hero=None):
    all_logs = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".xml"):
                file_path = os.path.join(dirpath, filename)
                try:
                    tree = ET.parse(file_path)
                    root_xml = tree.getroot()
                    if hero is None:
                        session_general = root_xml.find('general')
                        if session_general is not None and session_general.find('nickname') is not None:
                            hero = session_general.find('nickname').text.strip()
                    logs = parse_decision_logs(root_xml, hero)
                    all_logs.extend(logs)
                    print(f"Processed file: {file_path} -> {len(logs)} snapshots.")
                except ET.ParseError as e:
                    print(f"Error parsing XML file: {file_path}", e)
    return all_logs

# ------------------------------
# Main
# ------------------------------

if __name__ == '__main__':
    root_folder = "ipoker_hh_test"  # Adjust the folder path as needed
    hero_name = None  # Or manually set your hero's name
    print("Processing all hand history XML files in folder:", root_folder)
    all_logs = process_all_hand_history(root_folder, hero=hero_name)
    print("Total snapshots extracted:", len(all_logs))
    with open("logs.json", "w") as outfile:
        json.dump(all_logs, outfile, indent=4)
    print("Data extraction complete. Saved to logs.json")


Processing all hand history XML files in folder: ipoker_hh_test
Processed file: ipoker_hh_test/Tournaments/7733651957.xml -> 38 snapshots.
Processed file: ipoker_hh_test/Tournaments/4980566349.xml -> 31 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978995976.xml -> 86 snapshots.
Processed file: ipoker_hh_test/Tournaments/7736176979.xml -> 38 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978983872.xml -> 80 snapshots.
Processed file: ipoker_hh_test/Tournaments/4968024165.xml -> 51 snapshots.
Processed file: ipoker_hh_test/Tournaments/4977840702.xml -> 59 snapshots.
Processed file: ipoker_hh_test/Tournaments/4973790181.xml -> 85 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978600192.xml -> 53 snapshots.
Processed file: ipoker_hh_test/Tournaments/4979832638.xml -> 100 snapshots.
Processed file: ipoker_hh_test/Tournaments/4977072864.xml -> 66 snapshots.
Processed file: ipoker_hh_test/Tournaments/4976476504.xml -> 75 snapshots.
Processed file: ipoker_hh_test/Tour

## Extract data from hand history folder, make the player name into number

In [8]:
#!/usr/bin/env python3
import os
import xml.etree.ElementTree as ET
import json
import re

# ------------------------------
# Helper functions
# ------------------------------

def safe_float(text):
    try:
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def simplify_action(action_details, round_no, blinds, pot_before_action):
    """
    Simplify the action into a single string category.
    
    Allowed original types:
      - 0 → fold
      - 3 → call
      - 4 → check
      - 5, 7, 23 → raise, which are further categorized as follows:
          * Preflop (round 1), using the big blind as reference:
              2.0×BB to 2.3×BB → "small raise preflop"
              2.3×BB to 3.0×BB → "mid raise preflop"
              3.0×BB to 4.0×BB → "big raise preflop"
              ≥ 4.0×BB         → "all in preflop"
          * Postflop (round > 1), using the pot (before the action) as reference:
              < 35% of pot      → "small raise postflop"
              35% to 70% of pot  → "mid raise postflop"
              70% to 110% of pot → "big raise postflop"
              ≥ 110% of pot     → "all in postflop"
    """
    allowed_types = {0: "fold", 3: "call", 4: "check", 5: "raise", 7: "raise", 23: "raise"}
    orig_type = action_details['action_type']
    if orig_type not in allowed_types:
        return None
    base_action = allowed_types[orig_type]
    new_action = action_details.copy()
    if base_action != "raise":
        new_action["simple_action_type"] = base_action
    else:
        if round_no == 1:
            bb = blinds.get("big_blind", 1)
            ratio = action_details['action_sum'] / bb if bb != 0 else 0
            if ratio < 2.3:
                new_action["simple_action_type"] = "small raise preflop"
            elif 2.3 <= ratio < 3.0:
                new_action["simple_action_type"] = "mid raise preflop"
            elif 3.0 <= ratio < 4.0:
                new_action["simple_action_type"] = "big raise preflop"
            elif ratio >= 4.0:
                new_action["simple_action_type"] = "all in preflop"
            else:
                new_action["simple_action_type"] = "raise"
        else:
            ratio = action_details['action_sum'] / pot_before_action if pot_before_action > 0 else 0
            if ratio < 0.35:
                new_action["simple_action_type"] = "small raise postflop"
            elif ratio < 0.70:
                new_action["simple_action_type"] = "mid raise postflop"
            elif ratio < 1.10:
                new_action["simple_action_type"] = "big raise postflop"
            else:
                new_action["simple_action_type"] = "all in postflop"
    return new_action

def parse_decision_logs(root, hero):
    logs = []
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        blinds = {
            'small_blind': safe_float(general.findtext('smallblind', default="0")),
            'big_blind': safe_float(general.findtext('bigblind', default="0")),
            'ante': safe_float(general.findtext('ante', default="0"))
        }
        
        # Get players element and determine initial stacks.
        players_elem = general.find('players')
        # Identify small blind and big blind using round 0 actions.
        small_blind_player = None
        big_blind_player = None
        for r in game.findall('round'):
            if int(r.attrib.get('no', 0)) == 0:
                for child in r:
                    if child.tag == "action":
                        if child.attrib.get('type') == "1":
                            small_blind_player = child.attrib.get('player')
                        elif child.attrib.get('type') == "2":
                            big_blind_player = child.attrib.get('player')
                break
        # Identify the button player from the dealer flag.
        button_player = None
        for p in players_elem.findall('player'):
            if p.attrib.get('dealer','0') == '1':
                button_player = p.attrib.get('name')
                break
        
        # Create mapping: small blind → 0, big blind → 1, button → 2.
        player_positions = {}
        if small_blind_player is not None:
            player_positions[small_blind_player] = 0
        if big_blind_player is not None:
            player_positions[big_blind_player] = 1
        if button_player is not None:
            player_positions[button_player] = 2
        
        # Get initial stacks for the mapped players.
        player_stacks = {}
        for p in players_elem.findall('player'):
            name = p.attrib['name']
            if name in player_positions:
                player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # simplified actions (each includes "action_round")
        snapshot_action_counter = 0
        
        # Track cumulative contributions from each player.
        player_contributions = {name: 0.0 for name in player_positions}
        
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum')),
                        'action_round': round_no
                    }
                    player = action_details['player']
                    contribution = action_details['action_sum']
                    
                    # For round 0 (blinds/antes): update contributions and pot; no snapshot.
                    if round_no < 1:
                        if player in player_contributions:
                            player_contributions[player] += contribution
                        pot_size += contribution
                        continue
                        
                    # For rounds ≥ 1, capture current pot (before adding current action).
                    current_pot = pot_size
                    # Actor's current stack (as seen when taking the action).
                    actor_current_stack = player_stacks.get(player, 0.0) - player_contributions.get(player, 0.0)
                    
                    # Simplify the action.
                    simple_action = simplify_action(action_details, round_no, blinds, current_pot)
                    
                    if simple_action is not None:
                        snapshot_action_counter += 1
                        simple_action['action_no'] = snapshot_action_counter
                        # Replace the "player" field with its numeric value.
                        numeric_player = player_positions.get(player)
                        simple_action["player"] = numeric_player
                        
                        # Build current stacks keyed by numeric positions.
                        current_player_stacks = { player_positions[name] : (player_stacks[name] - player_contributions.get(name, 0.0))
                                                 for name in player_positions }
                        
                        # Create snapshot only for non-hero actions.
                        if player != hero:
                            snapshot = {
                                "gamecode": gamecode,
                                "round_no": round_no,
                                "current_street": street_from_round(round_no),
                                "blinds": blinds,
                                "player_positions": player_positions,  # mapping from name to number
                                "player_stacks": current_player_stacks,  # mapping from number to current stack
                                "pot_size": current_pot,
                                "board_cards": board_cards.copy(),
                                "previous_actions": [],
                                "action": simple_action.copy(),
                                "players_remaining": sum(1 for v in active_players.values() if v),
                                "is_button": (player_positions.get(player) == 2),
                                "actor_hole_cards": get_hole_cards(game, player),
                                "actor_stack_size": actor_current_stack,
                                "actor_position": player_positions.get(player)
                            }
                            # For previous actions, use the numeric "player" already set.
                            for act in cumulative_actions:
                                act2 = act.copy()
                                # Since act["player"] is now numeric, we simply assign it.
                                act2["player_position"] = act["player"]
                                snapshot["previous_actions"].append(act2)
                            logs.append(snapshot)
                    
                    # Update contributions and pot AFTER snapshot creation.
                    if player in player_contributions:
                        player_contributions[player] += contribution
                    pot_size += contribution
                    
                    if simple_action is not None:
                        cumulative_actions.append(simple_action)
                    
                    # Mark a player as inactive if they folded.
                    if action_details.get('action_type') == 0:
                        active_players[player] = False
    return logs

# ------------------------------
# Process All XML Files in ipoker_hh Folder
# ------------------------------

def process_all_hand_history(root_folder, hero=None):
    all_logs = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".xml"):
                file_path = os.path.join(dirpath, filename)
                try:
                    tree = ET.parse(file_path)
                    root_xml = tree.getroot()
                    if hero is None:
                        session_general = root_xml.find('general')
                        if session_general is not None and session_general.find('nickname') is not None:
                            hero = session_general.find('nickname').text.strip()
                    logs = parse_decision_logs(root_xml, hero)
                    all_logs.extend(logs)
                    print(f"Processed file: {file_path} -> {len(logs)} snapshots.")
                except ET.ParseError as e:
                    print(f"Error parsing XML file: {file_path}", e)
    return all_logs

# ------------------------------
# Main
# ------------------------------

if __name__ == '__main__':
    root_folder = "ipoker_hh_test"  # Adjust folder path as needed.
    hero_name = None  # Or manually set your hero's name.
    print("Processing all hand history XML files in folder:", root_folder)
    all_logs = process_all_hand_history(root_folder, hero=hero_name)
    print("Total snapshots extracted:", len(all_logs))
    with open("logs.json", "w") as outfile:
        json.dump(all_logs, outfile, indent=4)
    print("Data extraction complete. Saved to logs.json")


Processing all hand history XML files in folder: ipoker_hh_test
Processed file: ipoker_hh_test/Tournaments/7733651957.xml -> 38 snapshots.
Processed file: ipoker_hh_test/Tournaments/4980566349.xml -> 31 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978995976.xml -> 86 snapshots.
Processed file: ipoker_hh_test/Tournaments/4968678455.xml -> 72 snapshots.
Processed file: ipoker_hh_test/Tournaments/4972830349.xml -> 76 snapshots.
Processed file: ipoker_hh_test/Tournaments/4973069019.xml -> 94 snapshots.
Processed file: ipoker_hh_test/Tournaments/7736176979.xml -> 38 snapshots.
Processed file: ipoker_hh_test/Tournaments/4970969451.xml -> 19 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978983872.xml -> 80 snapshots.
Processed file: ipoker_hh_test/Tournaments/4968024165.xml -> 51 snapshots.
Processed file: ipoker_hh_test/Tournaments/4977840702.xml -> 59 snapshots.
Processed file: ipoker_hh_test/Tournaments/4973790181.xml -> 85 snapshots.
Processed file: ipoker_hh_test/Tourn

## Extract data from hand history folder(GPT o3 mini version)

In [6]:
#!/usr/bin/env python3
import os
import xml.etree.ElementTree as ET
import json
import re

# ------------------------------
# Helper functions
# ------------------------------

def safe_float(text):
    try:
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def simplify_action(action_details, round_no, blinds, pot_before_action):
    """
    Simplify the action into a single string category.
    
    Allowed original types:
      - 0 → fold
      - 3 → call
      - 4 → check
      - 5, 7, 23 → raise, which are further categorized as follows:
          * Preflop (round 1), using the big blind as reference:
              2.0×BB to 2.3×BB → "small raise preflop"
              2.3×BB to 3.0×BB → "mid raise preflop"
              3.0×BB to 4.0×BB → "big raise preflop"
              ≥ 4.0×BB         → "all in preflop"
          * Postflop (round > 1), using the pot (before the action) as reference:
              < 35% of pot      → "small raise postflop"
              35% to 70% of pot  → "mid raise postflop"
              70% to 110% of pot → "big raise postflop"
              ≥ 110% of pot     → "all in postflop"
    """
    allowed_types = {0: "fold", 3: "call", 4: "check", 5: "raise", 7: "raise", 23: "raise"}
    orig_type = action_details['action_type']
    if orig_type not in allowed_types:
        return None
    base_action = allowed_types[orig_type]
    new_action = action_details.copy()
    if base_action != "raise":
        new_action["simple_action_type"] = base_action
    else:
        if round_no == 1:
            bb = blinds.get("big_blind", 1)
            ratio = action_details['action_sum'] / bb if bb != 0 else 0
            if ratio <= 2.5:
                new_action["simple_action_type"] = "small raise preflop"
            elif ratio > 2.5:
                new_action["simple_action_type"] = "all in preflop"
            else:
                new_action["simple_action_type"] = "raise"
        else:
            ratio = action_details['action_sum'] / pot_before_action if pot_before_action > 0 else 0
            if ratio < 0.53:
                new_action["simple_action_type"] = "small raise postflop"
            else:
                new_action["simple_action_type"] = "big raise postflop"
    return new_action

def parse_decision_logs(root, hero):
    logs = []
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        blinds = {
            'small_blind': safe_float(general.findtext('smallblind', default="0")),
            'big_blind': safe_float(general.findtext('bigblind', default="0")),
            'ante': safe_float(general.findtext('ante', default="0"))
        }
        
        # Get players element and determine initial stacks.
        players_elem = general.find('players')
        # Identify small blind and big blind using round 0 actions.
        small_blind_player = None
        big_blind_player = None
        for r in game.findall('round'):
            if int(r.attrib.get('no', 0)) == 0:
                for child in r:
                    if child.tag == "action":
                        if child.attrib.get('type') == "1":
                            small_blind_player = child.attrib.get('player')
                        elif child.attrib.get('type') == "2":
                            big_blind_player = child.attrib.get('player')
                break
        # Identify the button player from the dealer flag.
        button_player = None
        for p in players_elem.findall('player'):
            if p.attrib.get('dealer','0') == '1':
                button_player = p.attrib.get('name')
                break
        
        # Create mapping: small blind → 0, big blind → 1, button → 2.
        player_positions = {}
        if small_blind_player is not None:
            player_positions[small_blind_player] = 0
        if big_blind_player is not None:
            player_positions[big_blind_player] = 1
        if button_player is not None:
            player_positions[button_player] = 2
        
        # Get initial stacks for the mapped players.
        player_stacks = {}
        for p in players_elem.findall('player'):
            name = p.attrib['name']
            if name in player_positions:
                player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # simplified actions (each includes "action_round")
        snapshot_action_counter = 0
        
        # Track cumulative contributions from each player.
        player_contributions = {name: 0.0 for name in player_positions}
        
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum')),
                        'action_round': round_no
                    }
                    player = action_details['player']
                    contribution = action_details['action_sum']
                    
                    # For round 0 (blinds/antes): update contributions and pot; no snapshot.
                    if round_no < 1:
                        if player in player_contributions:
                            player_contributions[player] += contribution
                        pot_size += contribution
                        continue
                        
                    # For rounds ≥ 1, capture current pot (before adding current action).
                    current_pot = pot_size
                    # Actor's current stack (capped at 0 if contributions exceed chips).
                    actor_current_stack = max(0, player_stacks.get(player, 0.0) - player_contributions.get(player, 0.0))
                    
                    # Simplify the action.
                    simple_action = simplify_action(action_details, round_no, blinds, current_pot)
                    
                    if simple_action is not None:
                        snapshot_action_counter += 1
                        simple_action['action_no'] = snapshot_action_counter
                        # Replace the "player" field with its numeric value.
                        numeric_player = player_positions.get(player)
                        simple_action["player"] = numeric_player
                        
                        # Build current stacks keyed by numeric positions, capping at 0.
                        current_player_stacks = {
                            player_positions[name] : max(0, player_stacks[name] - player_contributions.get(name, 0.0))
                            for name in player_positions
                        }
                        
                        # Create snapshot only for non-hero actions.
                        if player != hero:
                            snapshot = {
                                "gamecode": gamecode,
                                "round_no": round_no,
                                "current_street": street_from_round(round_no),
                                "blinds": blinds,
                                "player_positions": player_positions,  # mapping from name to number
                                "player_stacks": current_player_stacks,  # mapping from number to current stack
                                "pot_size": current_pot,
                                "board_cards": board_cards.copy(),
                                "previous_actions": [],
                                "action": simple_action.copy(),
                                "players_remaining": sum(1 for v in active_players.values() if v),
                                "is_button": (player_positions.get(player) == 2),
                                "actor_hole_cards": get_hole_cards(game, player),
                                "actor_stack_size": actor_current_stack,
                                "actor_position": player_positions.get(player)
                            }
                            # For previous actions, use the numeric "player" already set.
                            for act in cumulative_actions:
                                act2 = act.copy()
                                act2["player_position"] = act["player"]
                                snapshot["previous_actions"].append(act2)
                            logs.append(snapshot)
                    
                    # Update contributions and pot AFTER snapshot creation.
                    if player in player_contributions:
                        player_contributions[player] += contribution
                    pot_size += contribution
                    
                    if simple_action is not None:
                        cumulative_actions.append(simple_action)
                    
                    # Mark a player as inactive if they folded.
                    if action_details.get('action_type') == 0:
                        active_players[player] = False
    return logs

# ------------------------------
# Process All XML Files in ipoker_hh Folder
# ------------------------------

def process_all_hand_history(root_folder, hero=None):
    all_logs = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".xml"):
                file_path = os.path.join(dirpath, filename)
                try:
                    tree = ET.parse(file_path)
                    root_xml = tree.getroot()
                    if hero is None:
                        session_general = root_xml.find('general')
                        if session_general is not None and session_general.find('nickname') is not None:
                            hero = session_general.find('nickname').text.strip()
                    logs = parse_decision_logs(root_xml, hero)
                    all_logs.extend(logs)
                    print(f"Processed file: {file_path} -> {len(logs)} snapshots.")
                except ET.ParseError as e:
                    print(f"Error parsing XML file: {file_path}", e)
    return all_logs

# ------------------------------
# Main
# ------------------------------

if __name__ == '__main__':
    root_folder = "ipoker_hh"  # Adjust folder path as needed.
    hero_name = None  # Or manually set your hero's name.
    print("Processing all hand history XML files in folder:", root_folder)
    all_logs = process_all_hand_history(root_folder, hero=hero_name)
    print("Total snapshots extracted:", len(all_logs))
    with open("logs.json", "w") as outfile:
        json.dump(all_logs, outfile, indent=4)
    print("Data extraction complete. Saved to logs.json")


Processing all hand history XML files in folder: ipoker_hh
Processed file: ipoker_hh\Tournaments\4967863818.xml -> 3 snapshots.
Processed file: ipoker_hh\Tournaments\4967864063.xml -> 32 snapshots.
Processed file: ipoker_hh\Tournaments\4967864817.xml -> 51 snapshots.
Processed file: ipoker_hh\Tournaments\4967966606.xml -> 33 snapshots.
Processed file: ipoker_hh\Tournaments\4967966682.xml -> 31 snapshots.
Processed file: ipoker_hh\Tournaments\4967966752.xml -> 42 snapshots.
Processed file: ipoker_hh\Tournaments\4967966844.xml -> 64 snapshots.
Processed file: ipoker_hh\Tournaments\4967967010.xml -> 4 snapshots.
Processed file: ipoker_hh\Tournaments\4967967313.xml -> 67 snapshots.
Processed file: ipoker_hh\Tournaments\4967967817.xml -> 53 snapshots.
Processed file: ipoker_hh\Tournaments\4967967920.xml -> 34 snapshots.
Processed file: ipoker_hh\Tournaments\4967968111.xml -> 59 snapshots.
Processed file: ipoker_hh\Tournaments\4967968756.xml -> 61 snapshots.
Processed file: ipoker_hh\Tournam

## Exam the output json file

In [4]:
import json

with open('logs.json', 'r') as f:
    data = json.load(f)
print("Number of snapshots:", len(data))
print("First snapshot:")
print(json.dumps(data[0], indent=4))
print(json.dumps(data[1], indent=4))
print(json.dumps(data[2], indent=4))
print(json.dumps(data[3], indent=4))
print(json.dumps(data[4], indent=4))


Number of snapshots: 1791831
First snapshot:
{
    "gamecode": "6115617928",
    "round_no": 1,
    "current_street": "preflop",
    "blinds": {
        "small_blind": 1000.0,
        "big_blind": 2000.0,
        "ante": 0.0
    },
    "player_positions": {
        "soleggiata033": 0,
        "terpitors": 1,
        "ZioSergio3674": 2
    },
    "player_stacks": {
        "0": 49000.0,
        "1": 48000.0,
        "2": 50000.0
    },
    "pot_size": 3000.0,
    "board_cards": [],
    "previous_actions": [],
    "action": {
        "player": 2,
        "action_type": 3,
        "action_sum": 2000.0,
        "action_round": 1,
        "simple_action_type": "call",
        "action_no": 1
    },
    "players_remaining": 3,
    "is_button": true,
    "actor_hole_cards": [
        "DQ",
        "DA"
    ],
    "actor_stack_size": 50000.0,
    "actor_position": 2
}
{
    "gamecode": "6115617928",
    "round_no": 1,
    "current_street": "preflop",
    "blinds": {
        "small_blind": 1000.

## Model training

In [ ]:
#!/usr/bin/env python3
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# ------------------------------
# PARAMETERS
# ------------------------------
MAX_SEQ_LEN = 10   # maximum number of previous actions to consider in order
# Define the vocabulary for previous action types
action_vocab = [
    'fold', 'call', 'check', 
    'small raise preflop', 'all in preflop',
    'small raise postflop', 'big raise postflop'
]
# Reserve 0 for padding; assign IDs starting at 1.
action_vocab_map = {act: i+1 for i, act in enumerate(action_vocab)}
VOCAB_SIZE = len(action_vocab_map) + 1  # plus one for padding (id=0)

# ------------------------------
# DATA EXTRACTION FUNCTIONS
# ------------------------------
def extract_tabular_features(snapshot):
    """
    Extract tabular (non-sequential) features from the snapshot.
    Excludes details of previous actions and current_street.
    """
    features = {}
    features['round_no'] = snapshot.get('round_no', -1)
    features['small_blind'] = snapshot.get('blinds', {}).get('small_blind', 0)
    features['big_blind'] = snapshot.get('blinds', {}).get('big_blind', 0)
    features['ante'] = snapshot.get('blinds', {}).get('ante', 0)
    features['pot_size'] = snapshot.get('pot_size', 0)
    features['actor_stack_size'] = snapshot.get('actor_stack_size', 0)
    features['actor_position'] = snapshot.get('actor_position', -1)
    features['players_remaining'] = snapshot.get('players_remaining', 0)
    features['is_button'] = int(snapshot.get('is_button', False))
    # Include player stacks for positions "0", "1", "2"
    player_stacks = snapshot.get('player_stacks', {})
    for key in ['0', '1', '2']:
        features[f'player_stack_{key}'] = player_stacks.get(key, 0)
    # Include details from the current action (except its type, which is our target)
    action = snapshot.get('action', {})
    features['action_player'] = action.get('player', -1)
    features['action_round'] = action.get('action_round', -1)
    features['action_no'] = action.get('action_no', -1)
    return features

def extract_sequence_features(snapshot, max_seq_len=MAX_SEQ_LEN, action_vocab_map=action_vocab_map):
    """
    Extract a sequence (ordered list) of previous actions.
    For each previous action, return a vector of:
      [player, action_round, action_no, simple_action_type_id]
    The sequence is padded or truncated to a fixed length.
    """
    seq = []
    prev_actions = snapshot.get('previous_actions', [])
    for act in prev_actions:
        p = act.get('player', 0)
        r = act.get('action_round', 0)
        a_no = act.get('action_no', 0)
        act_type = act.get('simple_action_type', 'none')
        act_type_id = action_vocab_map.get(act_type, 0)
        seq.append([p, r, a_no, act_type_id])
    # Pad with zeros or truncate to fixed length
    if len(seq) < max_seq_len:
        pad_length = max_seq_len - len(seq)
        seq = seq + [[0, 0, 0, 0]] * pad_length
    else:
        seq = seq[:max_seq_len]
    return seq

# ------------------------------
# LOAD DATA
# ------------------------------
with open("logs.json", "r") as f:
    logs = json.load(f)

# ------------------------------
# BUILD DATASETS
# ------------------------------
tab_features_list = []
seq_features_list = []
target_list = []

for snapshot in logs:
    tab_features_list.append(extract_tabular_features(snapshot))
    seq_features_list.append(extract_sequence_features(snapshot))
    target_list.append(snapshot.get('action', {}).get('simple_action_type', 'unknown'))

# Convert tabular features to a DataFrame.
df_tab = pd.DataFrame(tab_features_list)
# No longer one-hot encoding current_street because it was removed.
# You may one-hot encode other categorical features if needed.

# Convert sequence features to a NumPy array.
X_seq = np.array(seq_features_list, dtype=np.int32)

# Encode targets.
le = LabelEncoder()
y_encoded = le.fit_transform(target_list)
NUM_CLASSES = len(le.classes_)

# Inspect and save data.
print("First 5 rows of tabular features:")
print(df_tab.head())
print("\nTarget distribution:")
print(pd.Series(target_list).value_counts())

df_tab.to_csv("tabular_features.csv", index=False)
# Flatten sequence features for CSV export.
flattened_seq = []
for seq in X_seq:
    flat = {}
    for i, (p, r, a_no, act_type_id) in enumerate(seq):
        flat[f"seq_{i}_player"] = p
        flat[f"seq_{i}_action_round"] = r
        flat[f"seq_{i}_action_no"] = a_no
        flat[f"seq_{i}_act_type_id"] = act_type_id
    flattened_seq.append(flat)
df_seq = pd.DataFrame(flattened_seq)
df_seq.to_csv("sequence_features.csv", index=False)

# ------------------------------
# SPLIT DATA INTO TRAIN/TEST
# ------------------------------
X_tab_train, X_tab_test, X_seq_train, X_seq_test, y_train, y_test = train_test_split(
    df_tab.values, X_seq, y_encoded, test_size=0.2, random_state=42
)

X_tab_train = X_tab_train.astype('float32')
X_tab_test = X_tab_test.astype('float32')
X_seq_train = X_seq_train.astype('int32')
X_seq_test = X_seq_test.astype('int32')
y_train = np.array(y_train)
y_test = np.array(y_test)

# ------------------------------
# BUILD THE MODEL
# ------------------------------
# Tabular branch.
tab_input = Input(shape=(X_tab_train.shape[1],), name="tab_input")
x_tab = layers.Dense(64, activation='relu')(tab_input)
x_tab = layers.Dense(32, activation='relu')(x_tab)

# Sequence branch.
seq_input = Input(shape=(MAX_SEQ_LEN, 4), name="seq_input")
act_type_seq = layers.Lambda(lambda x: x[:, :, 3])(seq_input)
embed_dim = 8
act_type_embedded = layers.Embedding(input_dim=VOCAB_SIZE, output_dim=embed_dim, mask_zero=True)(act_type_seq)
other_seq = layers.Lambda(lambda x: x[:, :, :3])(seq_input)
seq_combined = layers.Concatenate(axis=-1)([act_type_embedded, other_seq])
seq_out = layers.LSTM(32)(seq_combined)

# Combine branches.
combined = layers.Concatenate()([x_tab, seq_out])
x = layers.Dense(32, activation='relu')(combined)
output = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=[tab_input, seq_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ------------------------------
# TRAIN THE MODEL
# ------------------------------
history = model.fit(
    [X_tab_train, X_seq_train], y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=128
)

# ------------------------------
# EVALUATE THE MODEL
# ------------------------------
loss, accuracy = model.evaluate([X_tab_test, X_seq_test], y_test, verbose=0)
print("\nTest Accuracy:", accuracy)

y_pred = np.argmax(model.predict([X_tab_test, X_seq_test]), axis=1)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# ------------------------------
# MAKE A PREDICTION (EXAMPLE)
# ------------------------------
sample_idx = 0
sample_tab = X_tab_test[sample_idx:sample_idx+1]
sample_seq = X_seq_test[sample_idx:sample_idx+1]
pred_prob = model.predict([sample_tab, sample_seq])
predicted_class = le.inverse_transform([np.argmax(pred_prob)])

# Print predicted probabilities in a readable format.
pred_dict = {action: prob for action, prob in zip(le.classes_, pred_prob[0])}
print("\nPredicted probabilities:")
for action, prob in pred_dict.items():
    print(f"  {action}: {prob:.2%}")
print("Predicted action:", predicted_class[0])


First 5 rows of tabular features:
   round_no  small_blind  big_blind  ante  pot_size  actor_stack_size  \
0         1         10.0       20.0   0.0      30.0             500.0   
1         1         10.0       20.0   0.0      70.0             490.0   
2         2         10.0       20.0   0.0      90.0             460.0   
3         1         10.0       20.0   0.0      30.0             490.0   
4         1         10.0       20.0   0.0      90.0             530.0   

   actor_position  players_remaining  is_button  player_stack_0  \
0               2                  3          1           490.0   
1               0                  3          0           490.0   
2               2                  2          1           490.0   
3               2                  3          1           450.0   
4               1                  2          0           450.0   

   player_stack_1  player_stack_2  action_player  action_round  action_no  
0           480.0           500.0              2

/opt/anaconda3/envs/tf/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/tf/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/tf/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


1/1 [==============================] - 0s 393ms/step

Predicted probabilities:
  all in postflop: 1.16%
  all in preflop: 9.50%
  big raise postflop: 1.84%
  big raise preflop: 1.47%
  call: 21.00%
  check: 22.17%
  fold: 25.21%
  mid raise postflop: 6.96%
  mid raise preflop: 0.30%
  small raise postflop: 3.69%
  small raise preflop: 6.71%
Predicted action: fold


In [ ]:
#!/usr/bin/env python3
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# ------------------------------
# PARAMETERS
# ------------------------------
MAX_SEQ_LEN = 10   # maximum number of previous actions to consider in order
# Define the vocabulary for previous action types (you can adjust this list as needed)
action_vocab = [
    'fold', 'call', 'check', 
    'small raise preflop', 'mid raise preflop', 'big raise preflop', 'all in preflop',
    'small raise postflop', 'mid raise postflop', 'big raise postflop', 'all in postflop'
]
# Reserve 0 for padding; assign IDs starting at 1.
action_vocab_map = {act: i+1 for i, act in enumerate(action_vocab)}
VOCAB_SIZE = len(action_vocab_map) + 1  # plus one for padding (id=0)

# ------------------------------
# DATA EXTRACTION FUNCTIONS
# ------------------------------

def extract_tabular_features(snapshot):
    """
    Extract tabular (non-sequential) features from the snapshot.
    Excludes details of previous actions (which we handle separately).
    """
    features = {}
    features['round_no'] = snapshot.get('round_no', -1)
    features['current_street'] = snapshot.get('current_street', 'unknown')
    features['small_blind'] = snapshot.get('blinds', {}).get('small_blind', 0)
    features['big_blind'] = snapshot.get('blinds', {}).get('big_blind', 0)
    features['ante'] = snapshot.get('blinds', {}).get('ante', 0)
    features['pot_size'] = snapshot.get('pot_size', 0)
    features['actor_stack_size'] = snapshot.get('actor_stack_size', 0)
    features['actor_position'] = snapshot.get('actor_position', -1)
    features['players_remaining'] = snapshot.get('players_remaining', 0)
    features['is_button'] = int(snapshot.get('is_button', False))
    
    # Include player stacks (positions "0", "1", "2")
    player_stacks = snapshot.get('player_stacks', {})
    for key in ['0', '1', '2']:
        features[f'player_stack_{key}'] = player_stacks.get(key, 0)
    
    # Include details from the current action (except its type, which is our target)
    action = snapshot.get('action', {})
    features['action_player'] = action.get('player', -1)
    features['action_round'] = action.get('action_round', -1)
    features['action_no'] = action.get('action_no', -1)
    
    return features

def extract_sequence_features(snapshot, max_seq_len=MAX_SEQ_LEN, action_vocab_map=action_vocab_map):
    """
    Extract a sequence (ordered list) of previous actions.
    For each previous action, we return a vector of four numbers:
      [player, action_round, action_no, simple_action_type_id]
    The sequence is padded (or truncated) to a fixed length.
    """
    seq = []
    prev_actions = snapshot.get('previous_actions', [])
    for act in prev_actions:
        p = act.get('player', 0)
        r = act.get('action_round', 0)
        a_no = act.get('action_no', 0)
        act_type = act.get('simple_action_type', 'none')
        # Convert the action type to an integer ID (0 if not in vocabulary)
        act_type_id = action_vocab_map.get(act_type, 0)
        seq.append([p, r, a_no, act_type_id])
    # Pad with zeros if needed (or truncate if longer than max_seq_len)
    if len(seq) < max_seq_len:
        pad_length = max_seq_len - len(seq)
        seq = seq + [[0, 0, 0, 0]] * pad_length
    else:
        seq = seq[:max_seq_len]
    return seq

# ------------------------------
# LOAD DATA
# ------------------------------
with open("logs.json", "r") as f:
    logs = json.load(f)

# ------------------------------
# BUILD DATASETS
# ------------------------------
tab_features_list = []
seq_features_list = []
target_list = []

# The target is the current action's simple_action_type.
for snapshot in logs:
    tab_features_list.append(extract_tabular_features(snapshot))
    seq_features_list.append(extract_sequence_features(snapshot))
    target_list.append(snapshot.get('action', {}).get('simple_action_type', 'unknown'))

# Convert tabular features to a DataFrame.
df_tab = pd.DataFrame(tab_features_list)
# One-hot encode categorical features in the tabular part (e.g. current_street)
df_tab = pd.get_dummies(df_tab, columns=['current_street'], drop_first=True)

# Convert the sequence features to a NumPy array.
X_seq = np.array(seq_features_list, dtype=np.int32)  # shape: (num_samples, MAX_SEQ_LEN, 4)

# Encode target labels using LabelEncoder.
le = LabelEncoder()
y_encoded = le.fit_transform(target_list)
NUM_CLASSES = len(le.classes_)

# For inspection: Print a few rows of tabular data and target distribution.
print("First 5 rows of tabular features:")
print(df_tab.head())
print("\nTarget distribution:")
print(pd.Series(target_list).value_counts())

# ------------------------------
# SPLIT DATA INTO TRAIN/TEST
# ------------------------------
X_tab_train, X_tab_test, X_seq_train, X_seq_test, y_train, y_test = train_test_split(
    df_tab.values, X_seq, y_encoded, test_size=0.2, random_state=42
)

# Convert tabular inputs to float32 and sequence inputs to int32
X_tab_train = X_tab_train.astype('float32')
X_tab_test = X_tab_test.astype('float32')
X_seq_train = X_seq_train.astype('int32')
X_seq_test = X_seq_test.astype('int32')
y_train = np.array(y_train)
y_test = np.array(y_test)

# ------------------------------
# BUILD THE MODEL
# ------------------------------
# Define the tabular input branch.
tab_input = Input(shape=(X_tab_train.shape[1],), name="tab_input")
x_tab = layers.Dense(64, activation='relu')(tab_input)
x_tab = layers.Dense(32, activation='relu')(x_tab)

# Define the sequence input branch.
# Input shape is (MAX_SEQ_LEN, 4) for each sample.
seq_input = Input(shape=(MAX_SEQ_LEN, 4), name="seq_input")
# We treat the fourth element (index 3) as the categorical "action type" for each previous action.
act_type_seq = layers.Lambda(lambda x: x[:, :, 3])(seq_input)  # shape: (batch, MAX_SEQ_LEN)
# Embed the action type.
embed_dim = 8
act_type_embedded = layers.Embedding(input_dim=VOCAB_SIZE, output_dim=embed_dim, mask_zero=True)(act_type_seq)
# The other three numeric features (player, action_round, action_no) remain as is.
other_seq = layers.Lambda(lambda x: x[:, :, :3])(seq_input)  # shape: (batch, MAX_SEQ_LEN, 3)
# Concatenate the embedded categorical features with the numeric ones.
seq_combined = layers.Concatenate(axis=-1)([act_type_embedded, other_seq])
# Pass through an LSTM to capture the sequence context.
seq_out = layers.LSTM(32)(seq_combined)

# Combine the two branches.
combined = layers.Concatenate()([x_tab, seq_out])
x = layers.Dense(32, activation='relu')(combined)
output = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=[tab_input, seq_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ------------------------------
# TRAIN THE MODEL
# ------------------------------
history = model.fit(
    [X_tab_train, X_seq_train], y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=128
)

# ------------------------------
# EVALUATE THE MODEL
# ------------------------------
loss, accuracy = model.evaluate([X_tab_test, X_seq_test], y_test, verbose=0)
print("\nTest Accuracy:", accuracy)

y_pred = np.argmax(model.predict([X_tab_test, X_seq_test]), axis=1)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# ------------------------------
# MAKE A PREDICTION (EXAMPLE)
# ------------------------------
sample_idx = 0
sample_tab = X_tab_test[sample_idx:sample_idx+1]
sample_seq = X_seq_test[sample_idx:sample_idx+1]
pred_prob = model.predict([sample_tab, sample_seq])
predicted_class = le.inverse_transform([np.argmax(pred_prob)])
print("\nPredicted probabilities:", pred_prob)
print("Predicted action:", predicted_class[0])


## KNN Model

In [18]:
#!/usr/bin/env python3
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

# ------------------------------
# PARAMETERS
# ------------------------------
MAX_SEQ_LEN = 10   # fixed length of sequence features
action_vocab = [
    'fold', 'call', 'check', 
    'small raise preflop', 'mid raise preflop', 'big raise preflop', 'all in preflop',
    'small raise postflop', 'mid raise postflop', 'big raise postflop', 'all in postflop'
]
action_vocab_map = {act: i+1 for i, act in enumerate(action_vocab)}
VOCAB_SIZE = len(action_vocab_map) + 1  # reserve 0 for padding

# ------------------------------
# DATA EXTRACTION FUNCTIONS
# ------------------------------
def extract_tabular_features(snapshot):
    """Extract basic non-sequential features (current_street removed)."""
    features = {}
    features['round_no'] = snapshot.get('round_no', -1)
    features['small_blind'] = snapshot.get('blinds', {}).get('small_blind', 0)
    features['big_blind'] = snapshot.get('blinds', {}).get('big_blind', 0)
    features['ante'] = snapshot.get('blinds', {}).get('ante', 0)
    features['pot_size'] = snapshot.get('pot_size', 0)
    features['actor_stack_size'] = snapshot.get('actor_stack_size', 0)
    features['actor_position'] = snapshot.get('actor_position', -1)
    features['players_remaining'] = snapshot.get('players_remaining', 0)
    features['is_button'] = int(snapshot.get('is_button', False))
    # Include player stacks for positions "0", "1", "2"
    player_stacks = snapshot.get('player_stacks', {})
    for key in ['0', '1', '2']:
        features[f'player_stack_{key}'] = player_stacks.get(key, 0)
    # Current action details (excluding the action type)
    action = snapshot.get('action', {})
    features['action_player'] = action.get('player', -1)
    features['action_round'] = action.get('action_round', -1)
    features['action_no'] = action.get('action_no', -1)
    return features

def extract_sequence_features(snapshot, max_seq_len=MAX_SEQ_LEN, action_vocab_map=action_vocab_map):
    """
    Extract the ordered list of previous actions as a sequence.
    Each action is represented as: [player, action_round, action_no, simple_action_type_id]
    """
    seq = []
    prev_actions = snapshot.get('previous_actions', [])
    for act in prev_actions:
        p = act.get('player', 0)
        r = act.get('action_round', 0)
        a_no = act.get('action_no', 0)
        act_type = act.get('simple_action_type', 'none')
        act_type_id = action_vocab_map.get(act_type, 0)
        seq.append([p, r, a_no, act_type_id])
    # Pad or truncate to fixed length
    if len(seq) < max_seq_len:
        pad_length = max_seq_len - len(seq)
        seq = seq + [[0, 0, 0, 0]] * pad_length
    else:
        seq = seq[:max_seq_len]
    return seq

# ------------------------------
# LOAD DATA
# ------------------------------
with open("logs.json", "r") as f:
    logs = json.load(f)

tab_features_list = []
seq_features_list = []
target_list = []
for snapshot in logs:
    tab_features_list.append(extract_tabular_features(snapshot))
    seq_features_list.append(extract_sequence_features(snapshot))
    target_list.append(snapshot.get('action', {}).get('simple_action_type', 'unknown'))

# Convert tabular features into a DataFrame.
df_tab = pd.DataFrame(tab_features_list)

# For the k–NN model, we need a 2D feature vector. We can flatten the sequence features.
# Flatten each sequence (which is MAX_SEQ_LEN x 4) into a 1D array.
flattened_seq = np.array([np.array(seq).flatten() for seq in seq_features_list], dtype=np.float32)

# Now, concatenate the tabular features with the flattened sequence.
X_all = np.concatenate([df_tab.values.astype(np.float32), flattened_seq], axis=1)

# Encode target labels.
le = LabelEncoder()
y_encoded = le.fit_transform(target_list)
NUM_CLASSES = len(le.classes_)

print("Target distribution:")
print(pd.Series(target_list).value_counts())

# ------------------------------
# SPLIT DATA INTO TRAIN/TEST
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_encoded, test_size=0.2, random_state=42
)

# ------------------------------
# TRAIN A k–NN MODEL
# ------------------------------
# Choose a k value (e.g. 50) – you can tune this.
knn = KNeighborsClassifier(n_neighbors=50, weights='distance')  
knn.fit(X_train, y_train)

# ------------------------------
# EVALUATE THE MODEL
# ------------------------------
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("k–NN Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# For a new situation, get the distribution over actions:
sample_idx = 0  # for example, the first test sample
sample = X_test[sample_idx:sample_idx+1]
prob_dist = knn.predict_proba(sample)[0]
# Map each action to its probability:
pred_dict = {action: prob for action, prob in zip(le.classes_, prob_dist)}
print("\nPredicted distribution for the sample situation:")
for action, prob in pred_dict.items():
    print(f"  {action}: {prob:.2%}")


Target distribution:
fold                    182754
check                   157837
call                    150630
all in preflop           66354
mid raise postflop       50246
small raise preflop      47482
small raise postflop     26299
big raise postflop       13047
big raise preflop        10483
all in postflop           8182
mid raise preflop         2204
Name: count, dtype: int64
k–NN Accuracy: 0.49355014534883723
Classification Report:
                      precision    recall  f1-score   support

     all in postflop       0.07      0.02      0.03      1658
      all in preflop       0.38      0.24      0.29     13284
  big raise postflop       0.15      0.06      0.09      2598
   big raise preflop       0.32      0.11      0.16      2045
                call       0.45      0.47      0.46     30178
               check       0.56      0.75      0.64     31678
                fold       0.52      0.60      0.56     36544
  mid raise postflop       0.43      0.28      0.33     1

## GPT o3 mini code for model training

In [5]:
#!/usr/bin/env python3
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# ------------------------------
# PARAMETERS and Vocabulary
# ------------------------------
MAX_SEQ_LEN = 10   # maximum number of previous actions to consider
action_vocab = [
    'fold', 'call', 'check', 
    'small raise preflop', 'mid raise preflop', 'big raise preflop', 'all in preflop',
    'small raise postflop', 'mid raise postflop', 'big raise postflop', 'all in postflop'
]
# Reserve 0 for padding; IDs start at 1.
action_vocab_map = {act: i+1 for i, act in enumerate(action_vocab)}
VOCAB_SIZE = len(action_vocab_map) + 1  # plus one for padding

# ------------------------------
# Data Extraction Functions
# ------------------------------
def extract_tabular_features(snapshot):
    """
    Extract tabular (non-sequential) features from a snapshot.
    """
    features = {}
    features['round_no'] = snapshot.get('round_no', -1)
    features['small_blind'] = snapshot.get('blinds', {}).get('small_blind', 0)
    features['big_blind'] = snapshot.get('blinds', {}).get('big_blind', 0)
    features['ante'] = snapshot.get('blinds', {}).get('ante', 0)
    features['pot_size'] = snapshot.get('pot_size', 0)
    features['actor_stack_size'] = snapshot.get('actor_stack_size', 0)
    features['actor_position'] = snapshot.get('actor_position', -1)
    features['players_remaining'] = snapshot.get('players_remaining', 0)
    features['is_button'] = int(snapshot.get('is_button', False))
    # Include stacks for positions "0", "1", "2"
    player_stacks = snapshot.get('player_stacks', {})
    for key in ['0', '1', '2']:
        features[f'player_stack_{key}'] = player_stacks.get(key, 0)
    # Current action details (except the type which is our target)
    action = snapshot.get('action', {})
    features['action_player'] = action.get('player', -1)
    features['action_round'] = action.get('action_round', -1)
    features['action_no'] = action.get('action_no', -1)
    return features

def extract_sequence_features(snapshot, max_seq_len=MAX_SEQ_LEN, action_vocab_map=action_vocab_map):
    """
    Extract a fixed-length sequence of previous actions.
    Each entry: [player, action_round, action_no, simple_action_type_id]
    """
    seq = []
    prev_actions = snapshot.get('previous_actions', [])
    for act in prev_actions:
        p = act.get('player', 0)
        r = act.get('action_round', 0)
        a_no = act.get('action_no', 0)
        act_type = act.get('simple_action_type', 'none')
        act_type_id = action_vocab_map.get(act_type, 0)
        seq.append([p, r, a_no, act_type_id])
    # Pad with zeros or truncate to fixed length.
    if len(seq) < max_seq_len:
        pad_length = max_seq_len - len(seq)
        seq = seq + [[0, 0, 0, 0]] * pad_length
    else:
        seq = seq[:max_seq_len]
    return seq

# ------------------------------
# Load and Prepare Data
# ------------------------------
with open("logs.json", "r") as f:
    logs = json.load(f)

tab_features_list = []
seq_features_list = []
target_list = []

for snapshot in logs:
    tab_features_list.append(extract_tabular_features(snapshot))
    seq_features_list.append(extract_sequence_features(snapshot))
    target_list.append(snapshot.get('action', {}).get('simple_action_type', 'unknown'))

df_tab = pd.DataFrame(tab_features_list)
X_seq = np.array(seq_features_list, dtype=np.int32)

# Encode target action classes.
le = LabelEncoder()
y_encoded = le.fit_transform(target_list)
NUM_CLASSES = len(le.classes_)

print("Tabular features sample:")
print(df_tab.head())
print("\nTarget distribution:")
print(pd.Series(target_list).value_counts())

# Save features (optional)
# df_tab.to_csv("tabular_features.csv", index=False)
# flattened_seq = []
# for seq in X_seq:
#     flat = {}
#     for i, (p, r, a_no, act_type_id) in enumerate(seq):
#         flat[f"seq_{i}_player"] = p
#         flat[f"seq_{i}_action_round"] = r
#         flat[f"seq_{i}_action_no"] = a_no
#         flat[f"seq_{i}_act_type_id"] = act_type_id
#     flattened_seq.append(flat)
# pd.DataFrame(flattened_seq).to_csv("sequence_features.csv", index=False)

# ------------------------------
# Split Data and Normalize Tabular Features
# ------------------------------
X_tab_train, X_tab_test, X_seq_train, X_seq_test, y_train, y_test = train_test_split(
    df_tab.values, X_seq, y_encoded, test_size=0.2, random_state=42
)

# Normalize tabular features.
scaler = StandardScaler()
X_tab_train = scaler.fit_transform(X_tab_train.astype('float32'))
X_tab_test = scaler.transform(X_tab_test.astype('float32'))

X_seq_train = X_seq_train.astype('int32')
X_seq_test = X_seq_test.astype('int32')
y_train = np.array(y_train)
y_test = np.array(y_test)

# ------------------------------
# Build the Model
# ------------------------------
# Tabular branch.
tab_input = Input(shape=(X_tab_train.shape[1],), name="tab_input")
x_tab = layers.Dense(64, activation='relu')(tab_input)
x_tab = layers.Dropout(0.3)(x_tab)
x_tab = layers.Dense(32, activation='relu')(x_tab)
x_tab = layers.Dropout(0.2)(x_tab)

# Sequence branch.
seq_input = Input(shape=(MAX_SEQ_LEN, 4), name="seq_input")
# Extract action type sequence (last element of each time step).
act_type_seq = layers.Lambda(lambda x: x[:, :, 3])(seq_input)
embed_dim = 8
act_type_embedded = layers.Embedding(input_dim=VOCAB_SIZE, output_dim=embed_dim, mask_zero=True)(act_type_seq)
# Extract the remaining features.
other_seq = layers.Lambda(lambda x: x[:, :, :3])(seq_input)
seq_combined = layers.Concatenate(axis=-1)([act_type_embedded, other_seq])
seq_out = layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2)(seq_combined)

# Combine both branches.
combined = layers.Concatenate()([x_tab, seq_out])
x = layers.Dense(32, activation='relu')(combined)
x = layers.Dropout(0.2)(x)
output = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=[tab_input, seq_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ------------------------------
# Train the Model with Early Stopping
# ------------------------------
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    [X_tab_train, X_seq_train], y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=128,
    callbacks=[early_stop]
)

# ------------------------------
# Evaluate the Model
# ------------------------------
loss, accuracy = model.evaluate([X_tab_test, X_seq_test], y_test, verbose=0)
print("\nTest Accuracy:", accuracy)

y_pred = np.argmax(model.predict([X_tab_test, X_seq_test]), axis=1)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# ------------------------------
# Example Prediction
# ------------------------------
sample_idx = 0
sample_tab = X_tab_test[sample_idx:sample_idx+1]
sample_seq = X_seq_test[sample_idx:sample_idx+1]
pred_prob = model.predict([sample_tab, sample_seq])
predicted_class = le.inverse_transform([np.argmax(pred_prob)])

pred_dict = {action: prob for action, prob in zip(le.classes_, pred_prob[0])}
print("\nPredicted probabilities:")
for action, prob in pred_dict.items():
    print(f"  {action}: {prob:.2%}")
print("Predicted action:", predicted_class[0])


c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

Tabular features sample:
   round_no  small_blind  big_blind  ante  pot_size  actor_stack_size  \
0         1       1000.0     2000.0   0.0    3000.0           50000.0   
1         1       1000.0     2000.0   0.0   55000.0           48000.0   
2         1       1000.0     2000.0   0.0   55000.0           48000.0   
3         1       1000.0     2000.0   0.0    3000.0           49000.0   
4         1       1000.0     2000.0   0.0    4000.0           48000.0   

   actor_position  players_remaining  is_button  player_stack_0  \
0               2                  3          1         49000.0   
1               1                  3          0             0.0   
2               2                  2          1             0.0   
3               0                  2          0         49000.0   
4               1                  2          0         48000.0   

   player_stack_1  player_stack_2  action_player  action_round  action_no  
0         48000.0         50000.0              2         

c:\Users\j4829\.conda\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\j4829\.conda\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

     all in postflop       0.00      0.00      0.00      4055
      all in preflop       0.45      0.08      0.13     34630
  big raise postflop       0.00      0.00      0.00      6770
   big raise preflop       0.00      0.00      0.00      5458
                call       0.47      0.56      0.51     73627
               check       0.67      0.93      0.78     78763
                fold       0.57      0.80      0.67     95859
  mid raise postflop       0.60      0.27      0.37     23959
   mid raise preflop       0.00      0.00      0.00      1574
small raise postflop       0.55      0.36      0.44     12932
 small raise preflop       0.50      0.02      0.03     20740

            accuracy                           0.57    358367
           macro avg       0.35      0.27      0.27    358367
        weighted avg       0.53      0.57      0.51    358367


Predicted probabilities:
  all in postflop: 0.75%
  all in preflop

c:\Users\j4829\.conda\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
